In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas_profiling as pp
from datetime import datetime
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
import eli5
from eli5.sklearn import PermutationImportance

sns.set_style('whitegrid')
pd.set_option('display.max_columns', None) # display all columns

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("train.csv", delimiter=";")

In [4]:
df.sample(3).T

,44882,33509,32653
id,44883,33510,32654
year,2019,2018,2018
month,1,11,11
day,24,20,15
day_part,2,3,4
start_hub_id,83d45ec7-c3df-4dd4-867c-bec9c1ff8835,b8e69830-2432-476c-8deb-686194b27434,06280891-a1ca-4131-9162-b95bf14067d1
weekday,4,2,4
week,3,47,46
checkouts,5,0,0
reservations,4,0,0


## some basic data cleaning

In [5]:
# drop features ID's
# higly correlated features between eachother can be removed
df = df.drop(['id', 'month', 'checkout_min12h'], axis=1)

## Create a training dataset

In [6]:
# target
y = df["checkouts"]

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50996 entries, 0 to 50995
Data columns (total 20 columns):
year                50996 non-null int64
day                 50996 non-null int64
day_part            50996 non-null int64
start_hub_id        50996 non-null object
weekday             50996 non-null int64
week                50996 non-null int64
checkouts           50996 non-null int64
reservations        50996 non-null int64
checkout_min24h     50996 non-null int64
checkout_min1w      50996 non-null int64
checkout_min2w      50996 non-null int64
checkout_min3w      50996 non-null int64
checkout_min4w      50996 non-null int64
temp                48816 non-null object
rain                48816 non-null object
is_holiday          50996 non-null bool
holidays_in_week    50996 non-null int64
zuid_school         50996 non-null bool
midden_school       50996 non-null bool
noord_school        50996 non-null bool
dtypes: bool(4), int64(13), object(3)
memory usage: 6.4+ MB


In [8]:
df.select_dtypes(exclude="object")

,year,day,day_part,weekday,week,checkouts,reservations,checkout_min24h,checkout_min1w,checkout_min2w,checkout_min3w,checkout_min4w,is_holiday,holidays_in_week,zuid_school,midden_school,noord_school
0,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
1,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
2,2018,14,2,1,20,1,1,0,0,0,0,0,False,1,False,False,False
3,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
4,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
5,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
6,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
7,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
8,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False
9,2018,14,2,1,20,0,0,0,0,0,0,0,False,1,False,False,False


In [9]:
# features
X = df.select_dtypes(exclude="object").drop(["checkouts"], axis=1)

## choose a model

### xgboost

In [10]:
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

In [11]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [12]:
xgboost = XGBRegressor(n_jobs=-1)

In [13]:
%%time
xgboost.fit(X_train, y_train)

NameError: name 'X_train' is not defined

In [14]:
xgboost.__dict__

{'max_depth': 3,
 'learning_rate': 0.1,
 'n_estimators': 100,
 'silent': True,
 'objective': 'reg:linear',
 'booster': 'gbtree',
 'gamma': 0,
 'min_child_weight': 1,
 'max_delta_step': 0,
 'subsample': 1,
 'colsample_bytree': 1,
 'colsample_bylevel': 1,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'base_score': 0.5,
 'missing': nan,
 'kwargs': {},
 '_Booster': None,
 'seed': None,
 'random_state': 0,
 'nthread': None,
 'n_jobs': -1,
 'importance_type': 'gain'}

In [15]:
predict = xgboost.predict(X_test)

NameError: name 'X_test' is not defined

In [16]:
np.sqrt(mean_squared_error(y_test, predict))

NameError: name 'y_test' is not defined

## let's try do cross_validation

In [17]:
n_splits = 10

In [18]:
#xgboost = XGBRegressor(n_jobs=-1)

In [19]:
params={'n_estimators':[10,20,30,100,300], 'max_depth':[5,10]}

In [20]:
# choose number of splits: (recommended: 10)
cv = StratifiedKFold(n_splits=n_splits, shuffle=True)

In [21]:
grid = GridSearchCV(XGBRegressor(objective="count:poisson"), param_grid=params, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1)

In [22]:
%%time
# calculate the best parameters
grid.fit(X, y)

CPU times: user 4.71 s, sys: 95.9 ms, total: 4.8 s
Wall time: 1min 42s


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
       error_score='raise-deprecating',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='count:poisson', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [10, 20, 30, 100, 300], 'max_depth': [5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [23]:
grid.__dict__

{'scoring': 'neg_mean_squared_error',
 'estimator': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
        colsample_bytree=1, gamma=0, importance_type='gain',
        learning_rate=0.1, max_delta_step=0, max_depth=3,
        min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
        nthread=None, objective='count:poisson', random_state=0,
        reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
        silent=True, subsample=1),
 'n_jobs': -1,
 'fit_params': None,
 'iid': 'warn',
 'refit': True,
 'cv': StratifiedKFold(n_splits=10, random_state=None, shuffle=True),
 'verbose': 0,
 'pre_dispatch': '2*n_jobs',
 'error_score': 'raise-deprecating',
 'return_train_score': 'warn',
 'param_grid': {'n_estimators': [10, 20, 30, 100, 300], 'max_depth': [5, 10]},
 'multimetric_': False,
 'best_index_': 3,
 'best_params_': {'max_depth': 5, 'n_estimators': 100},
 'best_score_': -0.4926809909108923,
 'best_estimator_': XGBRegressor(base_score=0.5, booster='gb

In [24]:
np.sqrt(abs(grid.best_score_))

0.701912381220685

## 30min after we received the test data (against leakage)

In [25]:
df_test = pd.read_csv('test.csv', delimiter=";")

In [26]:
df_test

,id,year,month,day,day_part,start_hub_id,weekday,week,reservations,checkout_min12h,checkout_min24h,checkout_min1w,checkout_min2w,checkout_min3w,checkout_min4w,temp,rain,is_holiday,holidays_in_week,zuid_school,midden_school,noord_school
0,50997,2019,2,28,1,48e4126f-361a-486d-86e1-e9b936fc4ed8,4,8,0,4,0,0,0,0,0,"3,5",0,False,0,True,True,False
1,50998,2019,2,28,1,5c825027-217b-48ce-a1e0-ee2224e2a1e6,4,8,0,2,0,0,0,0,0,"3,5",0,False,0,True,True,False
2,50999,2019,2,28,1,244c7355-6d54-48c7-943d-a4730f904787,4,8,0,2,0,0,0,0,0,7,0,False,0,True,True,False
3,51000,2019,2,28,1,a5b54f27-6fc8-40e9-89ab-91157908c1fe,4,8,0,0,0,0,0,0,0,"4,5",0,False,0,True,True,False
4,51001,2019,2,28,1,008290a4-3bf4-4a9f-b46a-27f840ead380,4,8,0,0,0,0,0,0,0,"4,5",0,False,0,True,True,False
5,51002,2019,2,28,1,b8e69830-2432-476c-8deb-686194b27434,4,8,3,0,0,0,0,0,0,"4,5",0,False,0,True,True,False
6,51003,2019,2,28,1,208d5007-e249-4ad5-a147-3a4203cfe96b,4,8,0,0,0,0,0,0,0,"4,5",0,False,0,True,True,False
7,51004,2019,2,28,1,83d45ec7-c3df-4dd4-867c-bec9c1ff8835,4,8,0,0,0,0,0,0,0,"6,5",0,False,0,True,True,False
8,51005,2019,2,28,1,4ae6e636-a294-483e-9da5-0b30470d2b5c,4,8,0,4,0,0,0,0,0,"4,5",0,False,0,True,True,False
9,51006,2019,2,28,1,2f740c8c-a9d5-48b6-9a4d-93cc0bd5b21c,4,8,0,2,0,0,2,0,0,"3,5",0,False,0,True,True,False


## clean the this testdata as well

In [27]:
df_test = df_test.drop(['id', 'month'], axis=1)

In [28]:
df_test.sample(3).T

,955,1160,653
year,2019,2019,2019
day,5,6,3
day_part,2,3,3
start_hub_id,48e4126f-361a-486d-86e1-e9b936fc4ed8,44cb2c4a-5887-4329-aafb-8977605eb4b6,7e7e1230-c0ca-4222-b924-1b2ed7fa89e4
weekday,2,3,0
week,9,9,8
reservations,0,0,0
checkout_min12h,0,0,0
checkout_min24h,1,0,0
checkout_min1w,6,0,0


In [40]:
y_test = df_test["checkout_min12h"]

In [41]:
# features
X_test = df_test.select_dtypes(exclude="object")

In [42]:
X_test.columns

Index(['year', 'day', 'day_part', 'weekday', 'week', 'reservations',
       'checkout_min12h', 'checkout_min24h', 'checkout_min1w',
       'checkout_min2w', 'checkout_min3w', 'checkout_min4w', 'is_holiday',
       'holidays_in_week', 'zuid_school', 'midden_school', 'noord_school'],
      dtype='object')

In [45]:
predictions = grid.predict(X_test.drop(["checkout_min12h"], axis=1))

In [46]:
predictions

array([0.01138693, 0.01138693, 0.01138693, ..., 1.5215889 , 0.73992985,
       0.06251732], dtype=float32)

In [47]:
y_test.shape

(1232,)

In [48]:
predictions.shape

(1232,)

In [53]:
np.round(predictions, 0)

array([0., 0., 0., ..., 2., 1., 0.], dtype=float32)

In [57]:
predictions.shape[0]

1232

In [58]:
np.ones(predictions.shape[0])

array([1., 1., 1., ..., 1., 1., 1.])

In [54]:
mean_squared_error(y_test, np.round(predictions, 0))

10.05275974025974

In [55]:
np.sqrt(np.abs(mean_squared_error(y_test, np.round(predictions, 0))))

3.170608733391703

In [59]:
np.sqrt(np.abs(mean_squared_error(y_test, np.round(np.ones(predictions.shape[0]), 0))))

2.285435250825165